In [67]:
#This IPYNB notebook processes data scraped the Mekong River Commission Website at https://portal.mrcmekong.org/time-series/ph
#We take the SVG Paths and find manually the coefficients used to draw the SVG, deriving the original value
#The formula is value= slope_coeff(max_height-point_height) since the y-axis goes from top_to_bottom
ls=['COD','DO','EC','NO3','N2','TSS','TEMP','PH']
lsraw={};
for i in ls:
    with open(i+'.txt','r') as f:
        s= f.read()
        print(i)
        lsraw[i]=[x.split() for x in s.split('L')]
        print(len(lsraw[i]))
        

COD
228
DO
220
EC
228
NO3
228
N2
228
TSS
222
TEMP
226
PH
228


In [68]:
#Diagnostic Script, checks when the x values are too far apart for a larger than 1 month gap
v='DO'
for i in range(1,len(lsraw[v])-1):
    xn,yn=lsraw[v][i+1]
    x,y= lsraw[v][i]
    xp,yp= lsraw[v][i-1]
    if abs((float(xn)-float(x))-(float(x)-float(xp))) >5:
        print(i, x)


17 101.61904761905
18 143.68186423506
33 233.11448834853
34 245.10435663627
35 251.00101317123
36 263.1874366768


In [69]:
#Populate Missing Data Temp
lsraw['TEMP'].insert(42,['Inserted', float(lsraw['TEMP'][41][1])/2+float(lsraw['TEMP'][42][1])/2])
lsraw['TEMP'].insert(40,['Inserted', float(lsraw['TEMP'][39][1])/2+float(lsraw['TEMP'][40][1])/2])

In [70]:
#Populate Missing Data TSS
v='TSS'
for k in range(6):
    lsraw[v].insert(i:=78,['Inserted', float(lsraw[v][i-1][1])/2+float(lsraw[v][i][1])/2])


In [71]:
#Populate Missing Data DO
v='DO'
lsraw[v].insert(i:=34,['Inserted', float(lsraw[v][i-1][1])/2+float(lsraw[v][i][1])/2])
lsraw[v].insert(i:=36,['Inserted', float(lsraw[v][i-1][1])/2+float(lsraw[v][i][1])/2])
for k in range(6):
    lsraw[v].insert(i:=18,['Inserted', float(lsraw[v][i-1][1])/2+float(lsraw[v][i][1])/2])



In [72]:
for k in lsraw.keys():
    print(k, len(lsraw[k]))

COD 228
DO 228
EC 228
NO3 228
N2 228
TSS 228
TEMP 228
PH 228


In [45]:
import pandas as pd

In [73]:
df= pd.DataFrame({k: 
             map(lambda vli: float(vli[1]),
                vl)
             for k, vl in lsraw.items()})

In [74]:
pd.date_range('2003-01-15', periods=228, freq='M')
df.insert(0, 'Date', pd.date_range('2003-01-15', periods=228, freq='M'))

In [76]:
lsval={'COD':14,'NO3':1.4,'N2':4,'TSS':350}
for i, v in lsval.items():
    df[i]=df[i].apply(lambda x: v/445 * (445-x))

In [78]:
df['EC']=df['EC'].apply(lambda x:0.0674157*(519.16-x))

In [80]:
df['DO']=df['DO'].apply(lambda x:8/445*(500.625-x))

In [82]:
df['PH']=df['PH'].apply(lambda x:32/6329*(1631.9-x))

In [84]:
df['TEMP']=df['TEMP'].apply(lambda x: 14/519 *(1260.478-x))

In [89]:
df

,Date,COD,DO,EC,NO3,N2,TSS,TEMP,PH
0,2003-01-31,2.94,5.78,13.119545,0.141,0.456,43.0,30.000049,7.241053
1,2003-02-28,1.82,6.61,14.929544,0.092,0.780,38.0,28.999728,7.081055
2,2003-03-31,1.68,7.19,17.459543,0.072,0.289,34.0,28.999728,7.191054
3,2003-04-30,1.71,6.98,17.299543,0.150,0.517,26.8,30.000049,7.551047
4,2003-05-31,2.30,7.02,24.199540,0.240,0.669,10.4,29.699952,6.701062
...,...,...,...,...,...,...,...,...,...
223,2021-08-31,3.81,5.24,14.579544,0.648,1.160,15.0,29.199792,7.201053
224,2021-09-30,4.09,4.47,28.199538,0.757,3.110,42.0,28.799663,6.961058
225,2021-10-31,3.50,5.50,10.089546,0.043,0.605,76.0,28.199471,6.571064
226,2021-11-30,3.37,5.29,20.149541,0.232,0.713,56.0,29.799985,6.471066
